In [ ]:
import os
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from pmdarima import pipeline, arima, model_selection
from sklearn.metrics import root_mean_squared_error
import datetime as dt

In [ ]:
file_P = os.path.join(os.getcwd(), 'Elspotprices2nd.csv')
df_prices = pd.read_csv(file_P)
df_prices['HourUTC'] = pd.to_datetime(df_prices['HourUTC'])

file_P = os.path.join(os.getcwd(), 'ProdConData.csv')
df_data = pd.read_csv(file_P)
df_data['HourUTC'] = pd.to_datetime(df_data['HourUTC'])


In [ ]:
df_prices["SpotPriceDKK"].max()

#mean spot price is 643.112
#lowest is -447.459
#highest is 6982.64

## 1.1

First, we must define the training dataset, which runs from 1/1/19 until 31/8/24, and the testing dataset, which runs from 1/9/24 until 30/9/24. For that we will visualize the data and then split it into the two groups.

**The provided data doesn't end on 30/09/2024, but on 31/12/202**

In [ ]:
#We define relevant timestamps to filter only for the time periods mentioned in the task for training and testing
#from start to end of testing data there are 720 data points

#Filtering time period for training and testing data
t_start = pd.Timestamp(dt.datetime(2019, 1, 1, 0, 0, 0))
t_end = pd.Timestamp(dt.datetime(2024, 9, 30, 23, 0, 0))


In [ ]:
#Data filtering -- data remains as dataframe with HourUTC and SpotPriceDKK; drop indices after narrowing data down to specified dates

reduced_df = df_prices.loc[(df_prices['HourUTC']>=t_start) & (df_prices['HourUTC']<=t_end)]
reduced_df = reduced_df.reset_index(drop=True)

#Data split 
train, test = model_selection.train_test_split(reduced_df["SpotPriceDKK"], test_size=720)

#n's are relevant for x 
n_train = len(train)
n_test = len(test)
n_data = len(reduced_df)

In [ ]:
#Data visualization
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1,n_train+1), train)
plt.plot(np.arange(n_train+1,n_data+1), test)
plt.legend(["Training set", "Testing set"])
plt.grid(alpha=0.25)
plt.xticks(np.arange(0, n_data+1, 365*24), rotation=45)
plt.tight_layout()
plt.show()

Then, day-ahead predictions will be done with a seasonal ARIMA model. 30 predictions are needed of 24 values each. The correct values for them are known--contained in the test dataset--, so the model will be updated after each forecast.

A persistence model is included in the graph to use as benchmark for the model. Each new set of 24 values will be assumed to be equal to the previous 24.

In [ ]:
# Create a pipeline with the appropriate m and k; m = 24 for daily seasonality and k = 4---k value is how smooth curve fitted is, and max should be m//2, so 12
pipe = pipeline.Pipeline([
    ("fourier", pm.preprocessing.FourierFeaturizer(m=24, k = 6)),
    ("arima", arima.AutoARIMA(stepwise=False, trace = False, error_action="ignore",
                              seasonal=False, maxiter=10, 
                              suppress_warnings=True))])

pipe.fit(train)

# Create a list for the forecasts
rolling_forecast = []
N = int(len(test)/24)

for i in range(N):
    forecast = pipe.predict(n_periods=24)
    pipe.update(test[i*24:(i+1)*24])
    rolling_forecast.extend(forecast)

# Plot forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1,n_train+1), train)
plt.plot(np.arange(n_train+1,n_data+1), test)
plt.plot(np.arange(n_train+1,n_data+1), rolling_forecast)
plt.title("24-hour ahead predictions")
plt.legend(["Training set", "Actual values", "Forecasts"])
plt.grid(alpha=0.25)
plt.xlim(n_data - 6*7*24, n_data)
plt.ylim(top=4000)
plt.tight_layout()
plt.show()

To compare the ARIMA with the persistance model we report the RMSE value of each.

In [ ]:
#plotting of benchmark,i.e daily persistence model

data_spotprices = reduced_df["SpotPriceDKK"].values

#Empty list for 24 hour predictions
Persistence24 = []

for i in range(N):
    Persistence24.extend(data_spotprices[len(train)+ 24 * (i - 1) : len(train) + i * 24])

# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1,n_train+1), train)
plt.plot(np.arange(n_train+1,n_data+1), test)
plt.plot(np.arange(n_train+1,n_data+1), rolling_forecast)
plt.plot(np.arange(n_train+1,n_data+1), Persistence24)
plt.title("24-hour ahead predictions")
plt.legend(["Training set", "Actual values", "Forecasts", "Daily persistence"])
plt.grid(alpha=0.25)
plt.xlim(n_data - 6*7*24, n_data)
plt.ylim(top=4000)
plt.xlabel("Time [hours]")
plt.ylabel("Spot price [DKK/MWh]")
plt.tight_layout()
plt.show()

#RMSE for ARIMA and persistence.
RMSE_P24 = root_mean_squared_error(Persistence24, test)
RMSE_F = root_mean_squared_error(rolling_forecast, test)

print("RMSE for daily persistence:", round(RMSE_P24))
print("RMSE for forecasts:", round(RMSE_F))
print(f"The length of the rolling forecast and the persistence are {len(rolling_forecast)} and {len(Persistence24)}, respectively")

#with original data and k = 6, RMSE are 365 persistence and 309 forecats; 309 is the lowest RMSE for the forecast model

## 1.2

We now add any exogenous variables to the model, creating an ARIMAX one. After testing different variable combinations, we opted for only two variables that had a positive correlation with the data.

In [ ]:
#Filtering the exogenous data to the same time period that we have for training and testing the price data.
reduced_df_data = df_data.loc[(df_data['HourUTC']>=t_start) & (df_data['HourUTC']<=t_end)]
reduced_df_data = reduced_df_data.reset_index(drop=True)
reduced_df_data = reduced_df_data.fillna(0) #with this I'm replacing all NaN values with 0

#Merging price and exogenous data
reduced_df_merged = pd.merge(reduced_df, reduced_df_data, on='HourUTC')

#Correlation analysis to observe effects on SpotPriceDKK
corr = reduced_df_merged.drop(columns=['HourUTC']).corr(method='pearson')
corr = corr.round(2)

In [ ]:
#creating an ARIMAX model; we already have train/test for price data
#we need X_train and X_test, which will contain the exogenous data

#spot price data split
train, test = model_selection.train_test_split(reduced_df["SpotPriceDKK"], test_size=720)

#n's are relevant for x 
n_train = len(train)
n_test = len(test)
n_data = len(reduced_df)

#choice 5 predictors and splitting the sets
X_train, X_test = model_selection.train_test_split(reduced_df_merged[['CentralPowerMWh', 'CommercialPowerMWh']], test_size=720)

#building the matrix for the ARIMAX model with the predictors
X_train_ar = np.column_stack([np.arange(1, n_train+1), X_train])

pipe = pipeline.Pipeline([
    ("fourier", pm.preprocessing.FourierFeaturizer(m=24, k = 6)),
    ("arima", arima.AutoARIMA(stepwise=False, trace=1, error_action="ignore",
                              seasonal=False, maxiter=10, 
                              suppress_warnings=True))])

pipe.fit(train, X = X_train_ar)

In [ ]:
rolling_forecast_multiple = []
N = int(len(test)/24)

for i in range(N):

    X_f = np.column_stack([np.arange(1, 24+1), 
                           X_test[i*24:(i+1)*24]])

    forecast = pipe.predict(n_periods=24, X = X_f)
    
    rolling_forecast_multiple.extend(forecast)

    pipe.update(test[i*24:(i+1)*24], X = X_f)

# Make any non-negative values equal to zero
rolling_forecast_multiple = [0 if x < 0 else x for x in rolling_forecast_multiple]

# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1,len(train)+1), train, color = "black")
plt.plot(np.arange(n_train + 1, n_data + 1), rolling_forecast_multiple, color = "blue")
plt.plot(np.arange(n_train+1,n_data+1), Persistence24, color = "green")
plt.plot(np.arange(n_train + 1, n_data + 1), test, color = "red")
plt.legend(["Training set", "Forecasted values", "Persistence", "Actual values"], loc = "upper left")
plt.grid(alpha=0.25)
plt.xlim(n_data - 6*7*24, n_data)
plt.ylim(top=4000)
plt.xlabel("Time [hours]")
plt.ylabel("Spot price [DKK/MWh]")
plt.tight_layout()
plt.show()

# Calculate the error metrics
RMSE_P24 = root_mean_squared_error(Persistence24, test)
RMSE_F2 = root_mean_squared_error(rolling_forecast_multiple, test)

print(f"RMSE for daily persistence: {RMSE_P24:.2f}")
print(f"RMSE for the new forecasts: {RMSE_F2:.2f}")

Relevant results of the correlation analysis:
    
- CentralPowerMWh (0.20)
- OffshoreWindLt100MW_MWh (-0.26)
- OnshoreWindLt50kW_MWh(-0.20)
- OnshoreWindGe50kW_MWh (-0.31)
- HydroPowerMWh (-0.24)
- PowerToHeatMWh (-0.16)
- CommercialPowerMWh (0.11)

Tested combinations of exogenous variables:

- Choice 1: OnshoreWindGe50kW_MWh (-0.31), OffshoreWindLt100MW_MWh (-0.26), HydroPowerMWh (-0.24)
- Choice 2: OnshoreWindGe50kW_MWh (-0.31), OffshoreWindLt100MW_MWh (-0.26) 
- Choice 3: OnshoreWindGe50kW_MWh (-0.31), HydroPowerMWh (-0.24)
- Choice 4: OnshoreWindGe50kW_MWh
- Choice 5: CentralPowerMWh (0.20), CommercialPowerMWh (0.11)
- Choice 6: CentralPowerMWh (0.20)

RMSE for choice 1: 314.79 at k=12

RMSE for choice 2: 313.83 at k=12, 312 at k=6, 317 at k=4

RMSE for choice 3: 315.59 at k=6

RMSE for choice 4: RMSE is *291.40 at k = 6*, 307.90 at k = 4, 309 at k = 8, 304 at k=7

RMSE for choice 5: 309 at k = 6
